In [1]:
path_train='/content/drive/MyDrive/deep project/dataset/train'
path_test='/content/drive/MyDrive/deep project/dataset/test'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout,MaxPooling2D
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
import glob as gb
from keras.applications.vgg16 import VGG16

In [3]:
code = {'Class (1)': 0, 'Class (2)': 1}

def getcode(n):
    for x, y in code.items():
        if n == y:
            return x

In [4]:
x_train = []
y_train = []
img_size = 220

for i in os.listdir(path_train):
    files = gb.glob(pathname=str(os.path.join(path_train, i, '*.png')))

    for j in tqdm(files):
        image = cv2.imread(j)
        image_array = cv2.resize(image, (img_size, img_size))
        x_train.append(list(image_array))
        y_train.append(code[i])

# Convert to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)
print(y_train.shape)

100%|██████████| 223/223 [00:03<00:00, 72.17it/s] 


(942, 220, 220, 3)
(942,)


In [5]:
x_test = []
y_test = []
img_size = 220
for i in os.listdir(path_test):
    files = gb.glob(pathname=str(os.path.join(path_test, i, '*.png')))

    for j in tqdm(files):
        image = cv2.imread(j)
        image_array = cv2.resize(image, (img_size, img_size))
        x_test.append(list(image_array))
        y_test.append(code[i])


# Convert to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)
print(x_test.shape)
print(y_test.shape)

100%|██████████| 53/53 [00:12<00:00,  4.40it/s]


(231, 220, 220, 3)
(231,)


In [6]:
model_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
model = Sequential()

for layer in model_vgg16.layers:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 220, 220, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 220, 220, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 110, 110, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 110, 110, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 110, 110, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 55, 55, 128)       0         
                                                         

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])


In [9]:
early_stopping_monitor = EarlyStopping(patience = 2)
model.fit(x_train,y_train,epochs = 10,batch_size = 32,callbacks = [early_stopping_monitor])

Epoch 1/10
30/30 [==============================] - ETA: 0s - loss: 4.1110 - accuracy: 0.8312

30/30 [==============================] - 19s 267ms/step - loss: 4.1110 - accuracy: 0.8312
Epoch 2/10
30/30 [==============================] - ETA: 0s - loss: 1.4100 - accuracy: 0.8970

30/30 [==============================] - 4s 118ms/step - loss: 1.4100 - accuracy: 0.8970
Epoch 3/10
30/30 [==============================] - ETA: 0s - loss: 0.5744 - accuracy: 0.9321

30/30 [==============================] - 4s 118ms/step - loss: 0.5744 - accuracy: 0.9321
Epoch 4/10
30/30 [==============================] - ETA: 0s - loss: 0.2199 - accuracy: 0.9607

30/30 [==============================] - 4s 119ms/step - loss: 0.2199 - accuracy: 0.9607
Epoch 5/10
30/30 [==============================] - ETA: 0s - loss: 0.1138 - accuracy: 0.9703

...

30/30 [==============================] - 4s 120ms/step - loss: 0.2188 - accuracy: 0.9607
Epoch 7/10
30/30 [==============================] - ETA: 0s - loss: 0.1392 - accuracy: 0.9820

30/30 [==============================] - 4s 120ms/step - loss: 0.1392 - accuracy: 0.9820
Epoch 8/10
30/30 [==============================] - ETA: 0s - loss: 0.0794 - accuracy: 0.9798

30/30 [==============================] - 4s 121ms/step - loss: 0.0794 - accuracy: 0.9798
Epoch 9/10
30/30 [==============================] - ETA: 0s - loss: 0.1044 - accuracy: 0.9788

30/30 [==============================] - 4s 122ms/step - loss: 0.1044 - accuracy: 0.9788
Epoch 10/10
30/30 [==============================] - ETA: 0s - loss: 0.1181 - accuracy: 0.9809

30/30 [==============================] - 4s 120ms/step - loss: 0.1181 - accuracy: 0.9809


In [10]:
model.evaluate(x_test,y_test.astype('float32'))

8/8 [==============================] - 4s 413ms/step - loss: 0.3907 - accuracy: 0.9177


[0.3907140791416168, 0.9177489280700684]